# Scaling up the database to monthly 

In [102]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [103]:
import sys
sys.path.append('./lib/')

import lib.notebookSetup

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%matplotlib inline

## Import data

In [ ]:
period = '20180210-3-1'

In [105]:
df = pd.read_csv('../data/output/03_scraped_data_clean_third_{}.csv'.format(period), 
                 sep='|', quoting=1, encoding='utf-8', low_memory=False)

In [106]:
df.shape

(1943782, 27)

In [107]:
df.head(1)

,entity,department,contract,year,month,group,lastn,lastn2,givenn,degree,qualif,position,region,specials,currency,salary,overt,start_date,end_date,obs,url,other,filename,salary1,yearmonth,datets,pd
0,Presidencia de la República,Presidencia de la República,Dotación de planta,año 2017,allyear,AUTORIDAD DE GOBIERNO,bachelet,jeria,veronica michelle,-1,MÉDICO,PRESIDENTA DE LA REPÚBLICA,RM,"2, 4, 8, 9",PESOS,10.168.106,No,11/03/2014,30/11/-1,-1,http://www.gobiernotransparentechile.cl/direct...,-1,../data/output/scraped_data_20180210-3.csv,10168106.0,2017-allyear,2017-01-01,False


## Filters 

### Rows with no salary 

In [108]:
print(df.shape)
df = df.loc[pd.notnull(df['salary1'])]
print(df.shape)
df = df.loc[df['salary1'] > 100]
print(df.shape)

(1943782, 27)
(1632784, 27)
(1631773, 27)


## Expanding Salary over time

### Define ID for person 

#### Create unique column

In [109]:
df['person'] = df['givenn'].fillna('-1') + ' ' + df['lastn'].fillna('-1') + ' ' + df['lastn2'].fillna('-1')
df['person'] = df['person'].str.replace('á', 'a').str.replace('é', 'e').str.replace('í', 'i').str.replace('ó', 'o').str.replace('ú', 'u')

# Remove numbers
#df['person'] = df['person'].str.replace('\d', '')

df['person'] = df['person'].str.replace('  ', ' ')
df['person'] = df['person'].str.title()
df['person'] = df['person'].str.strip()



people = list(df['person'].value_counts().index)
print('{:,} unique people'.format(len(people)))

86,130 unique people


#### Checks 

In [110]:
df.loc[df['person'] == '11 11 11']

,entity,department,contract,year,month,group,lastn,lastn2,givenn,degree,qualif,position,region,specials,currency,salary,overt,start_date,end_date,obs,url,other,filename,salary1,yearmonth,datets,pd,person


#### Standardize the uniques 

In [111]:
# Get list of unique IDs
ids = list(df['person'].unique())
#ids = [x.strip() for x in ids]
ids.sort()
#print(ids[0:100])
#ids = ids[0:100]

a, b = createLookupNormalize(ids, 94)

Sorting list...
Done sorting


A Jupyter Widget

In [117]:
# Bring standardized names to DF
print(df.shape)
dfnames = pd.DataFrame({'person' : a, 
                        'person_std' : b})
df = df.merge(dfnames, how='inner')
print(df.shape)

(1631773, 31)
(1631773, 29)


In [118]:
idcol = 'person_std'

### Timestamp columns 

In [119]:
df['datets'] = pd.to_datetime(df['datets'], format='%Y-%m-%d')
df['start1'] = pd.to_datetime(df['start_date'], format='%d/%m/%Y', errors='coerce')
df['end1'] = pd.to_datetime(df['end_date'], format='%d/%m/%Y', errors='coerce')
df['datets_raw'] = df['datets'].copy()

### Duplicated rows 

In [120]:
print(df.shape)
df = df.drop_duplicates([idcol, 'datets'])
print(df.shape)
df = df.dropna(subset=['datets'])
print(df.shape)

(1631773, 32)
(1446363, 32)
(1441473, 32)


### Sort by person and datets

In [122]:
df = df.sort_values([idcol, 'datets'])

### Coverage of datets

In [123]:
pd.notnull(df['datets']).value_counts()

True    1441473
Name: datets, dtype: int64

### Person categorical for speed 

In [124]:
df['personcat'] = df[idcol].astype('category')

### Order of columns

In [ ]:
masterList = list(df[idcol].value_counts().index)
cols = df.columns

dfs = []
failed = 0
df = df.sort_values([idcol, 'datets'])
for p in tqdm_notebook(masterList):
    try:
        aux = createSalaryTimeline(df, p)
        dfs.append(aux)
    except:
        failed = failed + 1
        print('{}: {}'.format(failed, p), end='')


A Jupyter Widget

./lib/functions.py:531: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

Exception in thread Thread-30:
Traceback (most recent call last):
  File "/Users/jaime/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/jaime/anaconda3/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/Users/jaime/anaconda3/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



In [ ]:
# Combine the data frames
dfss = pd.concat(dfs)

# Sort the columns
col2 = list(col) + [x for x in dfss.columns if x not in list(col)]
dfss = dfss.loc[:, col2]

## Checks of the output data frame

In [ ]:
dfss.shape

In [ ]:
dfss.head(1)

In [ ]:
pd.notnull(dfss['datets']).value_counts()

In [ ]:
dfss['person'].nunique()

In [ ]:
print('Unique people:', dfss[idcol].nunique())

### How much salary is paid per month? 

In [ ]:
start = datetime(2006,1,1)
end = datetime(2018,3,1)

In [ ]:
dfss.loc[(dfss['date'] > start) &
         (dfss['date'] < end)].groupby(pd.Grouper(key='date',freq='MS'))['salary1'].sum().plot()

### How many unique people seen per month? 

In [ ]:
dfss.loc[(dfss['date'] > start) &
         (dfss['date'] < end)].groupby(pd.Grouper(key='date',freq='MS'))[idcol].nunique().plot()

### Research a specific person

In [ ]:
dfss.loc[dfss['date'] < datetime(2012,1,1), ['entity', 'date', 'person', 'salary1']].sort_values(['date', 'salary1']).head()

In [ ]:
fig,ax = plt.subplots(figsize=(9,6))
dfss.loc[dfss['person'].str.contains('sergio ricardo trigo', na=False)].plot(x='date', y='salary1', ax=ax)
ax.set_ylim(0)
ax.set_xlim(datetime(2009,1,1))


# Check people on March 31st 2018 

In [ ]:
dfss.loc[dfss['date'] == datetime(2018,3,1), 'end_date'].value_counts()

In [ ]:
dfss.loc[(dfss['date'] == datetime(2018,3,1)) &
         (dfss['end_date'] == '30/11/-1')]

# Datets is null 

They are null because they were generated by the expanding algorithm

In [ ]:
dfss.loc[pd.isnull(dfss['datets'])].shape

# Export 

In [ ]:
dfss.to_csv('../data/output/04_scraped_data_clean_scaleup_{}.csv'.format(period), 
                 sep='|', encoding='utf-8', index=False)